In [1]:
# propitiatory invocation (i.e. the user hasn't installed oifits as a package)
import sys
sys.path.append('..') 

import numpy as np
from astropy.table import Table

## Opening OIFITS files

We import the oifits module and open sample OIFITS files.

In [2]:
import pyoifits as oifits

# OIFITS version 2 from GRAVITY @ VLTI
gravity = oifits.open('test1.fits')
# OIFITS version 1 from PIONIER @ VLTI
pionier = oifits.open('test2.fits')

# The GRAVITY pipeline produces a non-standard OI_FLUX table, we fix that.
for h in gravity[8::4]:
    h.rename_columns(FLUX='FLUXDATA')
    

Verify the compliance to the standard and fix mendable issues.  The typical offences are strings of inappropriate widths, bad float/double type, non-standard columns that should have an NS_ prefix, and units.

In [3]:
gravity.verify('fix+warn')
pionier.verify('fix+warn')

 [astropy.io.fits.verify]


The OIFITS instances print nicely, indicating extensions with the data dimension.

In [4]:
gravity

<OIFITS2 at 0x7fe94da477c0: <PrimaryHDU2 (void)> <ArrayHDU2 (6C×4R)> <TargetHDU2 (17C×2R)> <WavelengthHDU2 (2C×210R=210W)> <WavelengthHDU2 (2C×5R=5W)> <VisHDU2 (20C×6R×5W)> <Vis2HDU2 (12C×6R×5W)> <T3HDU2 (16C×4R×5W)> <FluxHDU1 (10C×4R×5W)> <VisHDU2 (29C×6R×210W)> <Vis2HDU2 (12C×6R×210W)> <T3HDU2 (16C×4R×210W)> <FluxHDU1 (15C×4R×210W)>>

In [5]:
pionier

<OIFITS1 at 0x7fe91ade6bd0: <PrimaryHDU1 (void)> <TargetHDU1 (17C×1R)> <WavelengthHDU1 (2C×3R=3W)> <ArrayHDU1 (5C×4R)> <Vis2HDU1 (10C×5R×3W)> <T3HDU1 (14C×4R×3W)>>

OIFITS files can be merged easily.  Reindexing of extensions and target/station indices are taken care of.

In [6]:
# we merge arrays and stations of the same name, not caring about their exact
# coordinates. Reason: PIONIER sets them to zero but we know both PIONIER and 
# GRAVITY name the stations correctly.
oifits.set_merge_settings(array_distance=1e+9, station_distance=1e+9)
merged = gravity + pionier
merged

<OIFITS2 at 0x7fe91ad16a90: <PrimaryHDU2 (void)> <TargetHDU2 (17C×3R)> <ArrayHDU2 (6C×7R)> <WavelengthHDU2 (2C×5R=5W)> <WavelengthHDU2 (2C×210R=210W)> <WavelengthHDU2 (2C×3R=3W)> <VisHDU2 (20C×6R×5W)> <VisHDU2 (29C×6R×210W)> <Vis2HDU2 (12C×6R×5W)> <Vis2HDU2 (12C×6R×210W)> <Vis2HDU2 (10C×5R×3W)> <T3HDU2 (16C×4R×5W)> <T3HDU2 (16C×4R×210W)> <T3HDU2 (14C×4R×3W)> <FluxHDU1 (10C×4R×5W)> <FluxHDU1 (15C×4R×210W)>>

In [7]:
merged[0].header[0:19]

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
DATE    = '2020-07-02T19:52:50' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
DATE-OBS= '2012-12-21T04:12:03.022' / Observing date                            
CONTENT = 'OIFITS2 '                                                            
REFERENC= 'MULTI   '                                                            
OBSERVER= 'UNKNOWN '           / Name of observer.                              
PROG_ID = 'MULTI   '                                                            
PROCSOFT= 'MULTI   '        

One of the problems of the OIFITS standard is how complicated the data structure is, with a lot of cross-references.  It is possible to obtain the data in a flat table, listing one single scalar observable per line together to all relevant parameters such as target, date, wavelength, band, baseline(s).

In [8]:
tab = merged.to_table()
print(tab['TARGET','EFF_WAVE','EFF_BAND','U1COORD','V1COORD','U2COORD','V2COORD','observable','value','error'])

 TARGET    EFF_WAVE    EFF_BAND  U1COORD ... observable  value    error  
-------- ----------- ----------- ------- ... ---------- ------- ---------
CO_Ori_A 2.02369e-06 8.50000e-08   5.488 ...     VISAMP 0.84688 0.0041681
CO_Ori_A 2.09294e-06 8.50000e-08   5.488 ...     VISAMP 0.94783 0.0019325
CO_Ori_A 2.19100e-06 8.50000e-08   5.488 ...     VISAMP 0.97285   0.00123
CO_Ori_A 2.29188e-06 8.50000e-08   5.488 ...     VISAMP  0.9789 0.0017413
CO_Ori_A 2.36233e-06 8.50000e-08   5.488 ...     VISAMP 0.96522 0.0013306
CO_Ori_A 2.02369e-06 8.50000e-08  -2.729 ...     VISAMP 0.72994 0.0063785
CO_Ori_A 2.09294e-06 8.50000e-08  -2.729 ...     VISAMP 0.80576 0.0026422
CO_Ori_A 2.19100e-06 8.50000e-08  -2.729 ...     VISAMP 0.86026 0.0021998
CO_Ori_A 2.29188e-06 8.50000e-08  -2.729 ...     VISAMP 0.89297 0.0013147
CO_Ori_A 2.36233e-06 8.50000e-08  -2.729 ...     VISAMP 0.92514 0.0019255
     ...         ...         ...     ... ...        ...     ...       ...
CO_Ori_B 2.42799e-06 4.40191e-09      

## Creating OIFITS files

We start by creating an array HDU.  The local station coordinates can be provided (East-North-up) instead of the XYZ geocentric format of the OIFITS. They will be converted (WGS84).  Nominal positions at VLTI have centimetre precision when compared to the ones provided by the metrology.

Note that neither PIONIER nor GRAVITY files above correctly enter the STA_XYZ keywords (respectively zero and East-North-up coordinates).  


In [9]:
# WGS coordinates of nominal VLTI centre  and nominal station positions.
# (InterfaceControl Document between VLTI and its Instruments (Part I)
# Document ESO-045686, v. 7.3, Sect. 3.3)
arrname = 'VLTI'
lat = -24.62743941
lon = -70.40498689
alt = 2669
# Stations East-North coordinates are given in
# https://www.eso.org/observing/etc/doc/viscalc/vltistations.html
sta_name = ['A0', 'B2', 'D0', 'C1']
tel_name = ['AT1', 'AT2', 'AT3', 'AT4']
diameter = [1.8, 1.8, 1.8, 1.8]
staenu_nom = np.array([[-14.642, -55.812, 4.54],
                       [  0.739, -75.899, 4.54],
                       [ 15.628, -45.397, 4.54],
                       [  5.691, -65.735, 4.54]])
# This differs fron the previous document by millimetres
# and is given by the VLTI metrology in the headers. It's also 
# (quite incorrectly) available in gravity.STAXYZ with West-South-up
# coordinates.
staenu =  gravity.get_arrayHDUs()[0].STAXYZ.copy()
staenu[:,0] = -staenu[:,0]
staenu[:,1] = -staenu[:,1]

# Determine the difference from nominal positions
dif = np.abs(staenu - staenu_nom)
print(f"Mean station position difference {1e3*np.mean(dif):.0f} mm, max {1e3*np.max(dif):.0f} mm")
# Build the OI_ARRAY table 
array  = oifits.new_array_hdu(arrname=arrname, lat=lat, lon=lon, alt=alt,
            tel_name=tel_name, sta_name=sta_name, staenu=staenu,
            diameter=1.8)

Mean station position difference 3 mm, max 7 mm


OI_TARGET HDUs can be created in the same way or directly from the SIMBAD identifiers. Non ascii-names will be substituted.

In [10]:
simbad_id = ['CO Ori', 'η Car']
category = ['SCI', 'CAL']
target = oifits.new_target_hdu_from_simbad(simbad_id, category=category)
Table(target.data)


TARGET_ID,RAEP0,DECEP0,EQUINOX,RA_ERR,DEC_ERR,SYSVEL,VELTYP,VELDEF,PMRA,PMDEC,PMRA_ERR,PMDEC_ERR,PARALLAX,PARA_ERR,TARGET,SPECTYP,CATEGORY
int16,float64,float64,float32,float64,float64,float64,str8,str8,float64,float64,float64,float64,float32,float32,str32,str32,str3
1,81.90973,11.4274786,2000.0,1.1250000033113692e-07,1.386111146873898e-07,23000.0,BARYCENT,OPTICAL,5.472222222222223e-07,-6.444444444444445e-07,3.0833333730697636e-08,4.6666666037506536e-08,6.388889e-07,1.3611111e-07,CO Ori,F7Ve,SCI
2,161.2647729375,-59.68443085,2000.0,2.777777777777778e-06,3.055555555555556e-06,-25000.0,BARYCENT,OPTICAL,-3.055555555555556e-06,1.138888888888889e-06,1.944444411330753e-07,2.222222255335914e-07,nan,nan,* eta Car,OBepec,CAL


Create OI_WAVELENGTH table

In [11]:
insname = 'TESTING-3CHANNELS'
wave = [2.0e-6, 2.2e-6, 2.4e-6]
band = [0.2e-6, 0.2e-6, 0.2e-6]
wavelength = oifits.new_wavelength_hdu(insname=insname, eff_wave=wave, eff_band=band)

Create sample OI_VIS2 table.  We let UCOORD and VCOORD to zero, as they will be updated using
MJD and OI_ARRAY data.

In [12]:
sta_index = [[4,3],[4,2],[4,1],[3,2],[3,1],[2,1]]
target_id = [1] * 6
vis2data = [[1, 1, 1]] * 6
vis2err = [[0.05, 0.05, 0.05]] * 6
mjd = gravity[-4].MJD

vis2 = oifits.new_vis2_hdu(
        insname=insname, arrname=arrname, mjd=mjd,
        ucoord=0, vcoord=0,
        target_id=target_id, sta_index=sta_index,
        vis2data=vis2data, vis2err=vis2err)

Create the OIFITS containing these tables. Update the (u, v) coordinates and verify it's well-formed.

In [13]:
obs = oifits.OIFITS2([oifits.PrimaryHDU2(), target, array, wavelength, vis2])
obs.update_uv()
obs.verify('fix+warn')
obs

FK5.ra[0]=<Longitude 81.90973932 deg> FK5.dec[0]=<Latitude 11.42746784 deg>
altaz.alt=<Latitude 46.68925205 deg> altaz.az=<Longitude 36.45447266 deg>
lon=314.1673839198693 lat=11.439030707449806


<OIFITS2 at 0x7fe91a7b0590: <PrimaryHDU2 (void)> <TargetHDU2 (18C×2R)> <ArrayHDU2 (5C×4R)> <WavelengthHDU2 (2C×3R=3W)> <Vis2HDU2 (10C×6R×3W)>>

Compare the (u, v) coordinates with those in the GRAVITY file. We've entered the same target, baselines, and MJD. We couldn't do it on `gravity` itself for it has a bogus STAXYZ column.  I have still no clue where the difference comes from (not atmospheric diffraction, probably not time).

In [34]:
gravity_uv = np.array([gravity[-4].UCOORD, gravity[-4].VCOORD]).T
uv = np.array([obs[-1].UCOORD, obs[-1].VCOORD]).T
dif = np.abs(gravity_uv - uv)
rdif = dif / np.sqrt(uv[:,[0]] ** 2 + uv[:,[1]] ** 2)
print(f"(u, v) difference          mean={dif.mean()*1e3:.0f} mm -- max={dif.max()*1e3:.0f} mm")
print(f"Relative (u, v) difference mean={rdif.mean():.1%}  -- max={rdif.max():.1%}")


(u, v) difference          mean=21 mm -- max=41 mm
Relative (u, v) difference mean=0.1%  -- max=0.2%
